In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import LinAlgError
from IPython.display import clear_output
from tqdm import tqdm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from rcnet import rc_activations
from mne.decoding import CSP
from mne import set_log_level as mne_set_log_level

mne_set_log_level('WARNING')
rng = np.random.default_rng(0)

In [2]:
nsubs = 163
nsamples = 191
nchannels = 139

dynamics_length = 20
approx_reservoir_size = 308
worker_id = 0
w_in_init = "classic"
normalization = None
nfilters = 10
nsplits = 10


labels = np.zeros(nsubs)
signals = np.zeros([nsubs, nchannels, nsamples])
for i in range(1, 98):
    name = 'filtered_data/HC_'+str(i)+'_det.dat'
    signal = np.loadtxt(name)
    signals[i-1] = signal.T
    labels[i-1] = 0
for i in range(98, 164):
    name = f"data/mdd_{i}_det.dat"
    signal = np.loadtxt(name)[1:]
    signal = np.delete(
        signal,
        np.arange(90, 115+1),
        axis=1
    )
    signals[i-1] = signal.T
    labels[i-1] = 1

leak = 0.4042626932016279
degree = 9
radius = 1.157000269488874

crossval = StratifiedShuffleSplit(n_splits=nsplits, test_size=0.2, random_state=0)
confusion_matrices = np.zeros([nsplits, 2, 2])
split_i = 0
for train_index, test_index in tqdm(crossval.split(X=np.zeros(nsubs), y=labels), desc="CV", total=nsplits):
    activations = np.zeros([nsubs, approx_reservoir_size, nsamples-dynamics_length-1])
    csp_fmri = CSP(n_components=14, transform_into="csp_space")
    csp_fmri.fit(X=signals[train_index], y=labels[train_index])
    signals_filtered = csp_fmri.transform(signals)

    for i in range(nsubs):
        activation = rc_activations(
            signals_filtered[i].T,
            dynamics_length,
            approx_reservoir_size,
            degree,
            radius,
            worker_id,
            w_in_init,
            normalization,
            leak
        )
        activations[i-1] = activation.T

    csp_activations = CSP(n_components=nfilters, transform_into="average_power", log=True)
    csp_activations = csp_activations.fit(X=activations[train_index], y=labels[train_index])
    activations_filtered = csp_activations.transform(activations)
    lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
    clf = lda.fit(activations_filtered[train_index, :nfilters], labels[train_index])
    truths = labels[test_index]
    predictions = clf.predict(activations_filtered[test_index])
    confusion_matrices[split_i] = confusion_matrix(y_true=truths, y_pred=predictions)
    split_i += 1

CV: 100%|██████████| 10/10 [00:37<00:00,  3.78s/it]


In [3]:
tn = confusion_matrices[:, 0, 0]
fn = confusion_matrices[:, 1, 0]
tp = confusion_matrices[:, 1, 1]
fp = confusion_matrices[:, 0, 1]
accuracy = (tp+tn) / (tp+tn+fp+fn)
recall = tp / (tp+fn)
precision = tp / (tp+fp)
f1 = 2 * (
    (precision*recall) / (precision+recall)
)

In [4]:
print(f"Accuracy: {np.mean(accuracy):.2f} ± {np.std(accuracy):.2f}")
print(f"Recall: {np.mean(recall):.2f} ± {np.std(recall):.2f}")
print(f"Precision: {np.mean(precision):.2f} ± {np.std(precision):.2f}")
print(f"F1-score: {np.mean(f1):.2f} ± {np.std(f1):.2f}")

Accuracy: 0.86 ± 0.06
Recall: 0.75 ± 0.11
Precision: 0.89 ± 0.09
F1-score: 0.81 ± 0.08
